## 簡介
語意相似是去判斷輸入的2個句子的意思偏向...想一下商業應用以及指標規劃，讓其完整。

In [ ]:
!pip install transformers

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import transformers

## 設定超參數



In [ ]:
max_length = 128
batch_size = 32
epochs = 2

labels = ["contradiction", "entailment", "neutral"]

In [ ]:
# 下載知名的SNLI 資料集並且壓縮

!curl -LO https://raw.githubusercontent.com/MohamadMerchant/SNLI/master/data.tar.gz
!tar -xvzf data.tar.gz

In [ ]:
# 原始資料集有超過55萬筆資訓練資料，我們只取部分10萬當做訓練

train_df = pd.read_csv("SNLI_Corpus/snli_1.0_train.csv", nrows=100000)
valid_df = pd.read_csv("SNLI_Corpus/snli_1.0_dev.csv")
test_df = pd.read_csv("SNLI_Corpus/snli_1.0_test.csv")

# 輸出資料的形狀
print(f"Total train samples: {train_df.shape}")
print(f"Total validation samples: {valid_df.shape}")
print(f"Total test samples: {test_df.shape}")

In [ ]:
# 簡單觀察資料

train_df.head()

In [ ]:
valid_df.head()

In [ ]:
test_df.head()

- 看到其similarity 為標籤，之後要轉換成整數
- sentence1 以及 sentence2為我們的輸入輸入，因此模型要使用RoBERTa，故會透過[SEP]將其隔開。

## 資料前處理
- 觀察資料缺失
- 觀察資料長度
- 觀察類別資訊
- 將資料包裝成適合的形式，可送入模型

In [ ]:
# 資料缺失值直接移除

print("遺失的資料比數")
print(train_df.isnull().sum())          # 會將每一個欄位的缺失比數顯示
train_df.dropna(axis=0, inplace=True)   # 將缺失值移除，且是inplace執行

In [ ]:
# 類別資訊比例(訓練)

train_df['similarity'].value_counts()

In [ ]:
# 類別資訊比例(測試)

valid_df['similarity'].value_counts()

> 比例是差不多的，就不多做處理。

但有-的類別，我們將其移除。

In [ ]:
# 移除-類別

train_df = train_df[train_df['similarity'] != '-']
valid_df = valid_df[valid_df['similarity'] != '-']
test_df = test_df[test_df['similarity'] != '-']

In [ ]:
# 將類別轉成數值，有兩種方法，直接轉換為index，或者one-hot encoding，這裡用index就好
mapping = {
    'neutral': 0,
    'contradiction': 1,
    'entailment': 2,
    0: 'neutral',
    1: 'contradiction',
    2: 'entailment'
}


train_df["label"] = train_df["similarity"].apply(
    lambda x: mapping[x]
)
# y_train = tf.keras.utils.to_categorical(train_df.label, num_classes=3)
y_train = train_df.label.values

valid_df["label"] = valid_df["similarity"].apply(
    lambda x: mapping[x]
)
# y_val = tf.keras.utils.to_categorical(valid_df.label, num_classes=3)
y_val = valid_df.label.values

test_df["label"] = test_df["similarity"].apply(
    lambda x: mapping[x]
)
# y_test = tf.keras.utils.to_categorical(test_df.label, num_classes=3)
y_test = test_df.label.values

In [ ]:
# 觀看label
train_df.head()

兩個句子連接的tokenizer

In [ ]:
tokenizer = transformers.RobertaTokenizer.from_pretrained('roberta-base')
tokenizer.encode_plus('You are my sunshine.', 'You are my heroes')

In [ ]:
# 建立一個 generator: 繼承 Sequence, https://www.tensorflow.org/api_docs/python/tf/keras/utils/Sequence


class BertSemanticDataGenerator(tf.keras.utils.Sequence):       # 至少要override __len__, __getitem__
    """
        產生 batch of data.
        

        Args:
            sentence_pairs: list/array, of s pairs.
            labels: list/array, of labels.
            batch_size: integer.
            shuffle: boolean, whether to shuffle the data.
            include_targets: boolean, whether to include the targets
        

        Returns:
            Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
            (or just `[input_ids, attention_mask, `token_type_ids]`
            if `include_targets=False`)
    """
    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=16,
        shuffle=True,
        include_targets=True
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets

        self.tokenizer = transformers.RobertaTokenizer.from_pretrained(
            "roberta-base", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()         # Method called at the end of every epoch.
    

    def __len__(self):
        return len(self.sentence_pairs) // self.batch_size
    

    def __getitem__(self, idx):
        index = self.indexes[idx*self.batch_size: (idx+1)*self.batch_size]
        sentence_pairs = self.sentence_pairs[self.indexes]

        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        input_ids = np.array(encoded['input_ids'], dtype='int32')
        attention_masks = np.array(encoded['attention_mask'], dtype='int32')
        token_type_ids = np.array(encoded['token_type_ids'], dtype='int32')

        if self.include_targets:
            labels = np.array(self.labels[index], dtype='int32')
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]
    

    def on_epoch_end(self):
        if self.shuffle:
            np.random.RandomState(41).shuffle(self.indexes)

建立模型

In [ ]:
# Create the model under a distribution strategy scope.
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Encoded token ids from BERT tokenizer.
    input_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )
    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )
    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )
    # Loading pretrained BERT model.
    bert_model = transformers.TFRobertaModel.from_pretrained("roberta-base")
    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = False

    bert_output = bert_model(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    sequence_output = bert_output.last_hidden_state     # 全部
    pooled_output = bert_output.pooler_output           # [CLS]
    # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
    bi_lstm = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, return_sequences=True)
    )(sequence_output)
    # Applying hybrid pooling approach to bi_lstm sequence output.
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
    concat = tf.keras.layers.concatenate([avg_pool, max_pool])
    dropout = tf.keras.layers.Dropout(0.3)(concat)
    output = tf.keras.layers.Dense(3, activation="softmax")(dropout)
    model = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=output
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="sparse_categorical_crossentropy",
        metrics=["acc"],
    )


print(f"Strategy: {strategy}")
model.summary()

In [ ]:
# 包裝好訓練資料

train_data = BertSemanticDataGenerator(
    train_df[["sentence1", "sentence2"]].values.astype("str"),
    y_train,
    batch_size=batch_size,
    shuffle=True,
)
valid_data = BertSemanticDataGenerator(
    valid_df[["sentence1", "sentence2"]].values.astype("str"),
    y_val,
    batch_size=batch_size,
    shuffle=False,
)

In [ ]:
# 確認資料無誤

for x in train_data:
    print(x)
    break

訓練模型

In [ ]:
transformers.logging.set_verbosity_error()      # 避免: Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
)

## Fine-tuning
This step must only be performed after the feature extraction model has been trained to convergence on the new data.

This is an optional last step where bert_model is unfreezed and retrained with a very low learning rate. This can deliver meaningful improvement by incrementally adapting the pretrained features to the new data.

In [ ]:
# Unfreeze the bert_model.
bert_model.trainable = True
# Recompile the model to make the change effective.
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
model.summary()

Fine tuning the model

In [ ]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)

準備預測資料

In [ ]:
test_data = BertSemanticDataGenerator(
    test_df[["sentence1", "sentence2"]].values.astype("str"),
    y_test,
    batch_size=batch_size,
    shuffle=False,
)
model.evaluate(test_data, verbose=1)

真正推論時使用的function

In [ ]:
def check_similarity(sentence1, sentence2):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )

    proba = model.predict(test_data[0])[0]
    idx = np.argmax(proba)
    proba = f"{proba[idx]: .2f}%"
    pred = labels[idx]
    return pred, proba

In [ ]:
sentence1 = "Two women are observing something together."
sentence2 = "Two women are standing with their eyes closed."
check_similarity(sentence1, sentence2)

In [ ]:
sentence1 = "A smiling costumed woman is holding an umbrella"
sentence2 = "A happy woman in a fairy costume holds an umbrella"
check_similarity(sentence1, sentence2)

In [ ]:
sentence1 = "A soccer game with multiple males playing"
sentence2 = "Some men are playing a sport"
check_similarity(sentence1, sentence2)

至此完成相似句子Roberta模型